## Eigenfaces (Linear Algebra Face Classification)
> All code is written in Python 3.12.1. Code may not perform as expected otherwise.

The dependancies of this notebook include: **numpy**, **mathplotlib**, **PIL**, **os**, and **pathlib**. \
This notebook seeks to implement methods described in the Turk and Pentland 1991 paper "Eigenfaces for Recognition".\
See: ["Eigenfaces for Recognition"](https://sites.cs.ucsb.edu/~mturk/Papers/mturk-CVPR91.pdf) by Matthew A. Turk and Alex P. Pentland at MIT.\
Additional resources: 
[[2]](https://doi.org/10.1016/j.protcy.2012.02.023, "A Face Recognition System Based on Eigenfaces Method by M.üge Çarıkçı & Figen Özen"), 
[[3]](https://www.ijstr.org/final-print/mar2020/Face-Recognition-By-Using-Eigen-Face-Method.pdf, "Face Recognition By Using Eigen Face Method
by V. Jalaja & G.S.G.N. Anjaneyulu"). \
It is significant to note that Turk and Pentland atrribute the motivation of using 'eigenfaces' to L. Sirovich and M. Kirby [[see]](https://opg.optica.org/josaa/fulltext.cfm?uri=josaa-4-3-519&id=2689, "Low-dimensional procedure for the characterization of human faces by L. Sirovich and M. Kirby"). 

> 1. Access and convert a set of test images into grayscale.
> 2. Convert the image from *N x N* (where *N* represents the h & w pixel magnitude) into a column vector with *N x 1* dimensionality.
> 3. 

In [ ]:
from pathlib import Path
from matplotlib import pyplot as plt  
import numpy as np
import cv2
import os

### 1.0. Vectorize Images and Load Into N x M Matrix
Every digital image can be defined as a set of pixels on a $N_a \times N_b$ grid (assuming single channel, grayscale not RGB). \
For the images which will be loaded as the training set, they have been pre-processed to be square such that $N_a = N_b = N$. \
As such each image is an $N \times N$ image ($N^2$ pixels), where $N = 128$ pixels.\
Digital images can be represented as a set of vectors by flattening the images to be of $N \times 1$ dimensionality.

Load all training images into a matrix, let the quantity of all training examples be $M$. \
Each image can be flattened, and added to a matrix horizontially to make a 'face space' (all the faces in a matrix).\
This new matrix, which we call $A$ will have the dimensionality $N^2 \times M$. \
We represent a single image, translated into a vector as the mathematical symbol gamma ($\Gamma_i$). \
The new matrix $A_0$ can be represented as the following: $A_o = [ \Gamma_1, \Gamma_2, ..., \Gamma_M]$.

We can use OpenCV2 (cv2) to load each of the images into numpy vectors, collated to make matrix $A_o$. \
OpenCV's imread takes a second parameter, which converts images into grayscale on load if provided with a zero. 

In [103]:
# Place your path here relative to this file
# Note: images need to be pre-processed to be of 128 x 128 pixel dimensionality.
PATH = str(Path.cwd()) + "\\faces"
IMAGE_NAMES = os.listdir(PATH)
M = len(IMAGE_NAMES)
N = 128 

# A_0 is the matrix containing all images, represented by column vectors
A_0 = []
for name in IMAGE_NAMES:
    image = cv2.imread(PATH + "\\" + name, 0)
    phi = np.asarray(image).reshape(1, N * N)[0]
    A_0.append(phi)
A_0 = np.array(A_0).T

### 1.2. Generate Mean/Average Face
Now lets talk about what this paper posits as a solution to recognizing faces within a frame (we will get to classification later).\
Instead of performing feature analysis specific regions of faces (i.e., recongition by distance between eyes and nose), \
Turk and Pentland propose the generation of a **mean** or **average** face, alongside a set of characteristics which deviate from \
the mean face (which they named **eigenfaces**), from which faces can be approximated or regenerated by a linear combination \
of eigenfaces with the average face.

So what? When implemented well, with enough training data, the model should be able to regenerate any image as a linear combination of eigenfaces \
(even if not a face). When regenerating images, weights are also calculated which are applied to each eigenface. Calculating the euclidian distances \
between a regenerated face, and a known face, can help us perform person recognition...however, we are getting ahead of ourselves here.

What is important to note here, is that we are interested in a **mean** face. \
We will figure out how to generate eigenfaces in the next section.

The mean face takes each column vector $\Gamma_i$ and averages each of its rows across the total sample size $M$. \
The symbol we use to represent the mean face is the mathematical symbol psi ($\psi$), and the average face is given by: \
$\psi = \frac{1}{M} \sum_{n = 1}^M \Gamma_i$, since we already have $A_o$, we can take the averages of the columns.\
Here is an example of what this looks like, just with a few Mark Zuckerberg images:
> 
> [<img src="source/average-face-example-zucc.jpg" width="550"/>](source/average-face-example-zucc.jpg)
> 

In [ ]:
# The mean face ψ
psi = A_0.mean(axis=1).reshape(N * N, 1)

**Optional:**
We can have a look at what the average face looks like. \
We'll first print out the matrix to see the values of each pixel, which represent grayscale intensity,\
Then we'll use mathplotlib to translate the matrix into an image. 

_You can also add print(psi) to see the individual pixel values which matplotlib is plotting._

In [ ]:
# Optional
plt.title('Mean face (psi, ψ) visualized'),plt.imshow(psi.reshape(N, N))

### 2.1. Normalize Face Space
Remember how we discussed regenerating faces via a linear combination of 'eigenfaces' with the mean face? \
We discussed how the 'eigenfaces' are related to discrepencies between faces. To progress towards this, \
we'll first need to grab the unique features of each image by subtracting the mean face from each training image. \
In other words, we'll remove the normalness from each image to be left with a unique ghost-like image. \
After we remove the normalness, we will have normalized the 'face space'.

We have previously defined $A_0$ as all the images in the training set, represented as column vectors, contained in a matrix.\
We have also defined psi as the mean face. Numpy allows us to perform element-wise arithmetic across two matricies,  
even if they have different dimensionalities, only if one is shared and the other is a multiple of the other. \
This is called __broadcasting__.

> Here's an example: \
> Let $A$ be a matrix of $1 \times 5$ dimensionality, and $B$ be a matrix of $12 \times 5$ dimensionality. \
> With Numpy, without making any changes to $A$ or $B$, we can perform the matrix operation $A + B$. \
> Numpy will take the matrix $A$ and duplicate it $12$ times horizontally to make it become a $12 \times 5$ matrix.\
> After broadcasting matrix $A$ to match the dimensionality of $B$, Numpy will perform element-wise addition.

This is incredibly useful in our case, as we can take the stored faces in $A_o$ of $N^2 \times M$ dimensionality, \
and exploit Numpy's broadcasting feature to subtract the mean face, with $N^2 \times 1$ dimensionality, from each \
image column vector.  

The set of all normalized face images is defined as $A = [\phi_1, \phi_2, ..., \phi_M]$.

In [123]:
A = A_0 - psi

> # In progress from this point on!

### 2.2. Principle Component Analysis
Here we enter the next stage of calculating our 'eigenfaces'. Here we'll perform something called \
Principle Component Analysis (PCA) or Karhunen-Loeve expansion (some resources: [[1]](https://www.geeksforgeeks.org/principal-component-analysis-pca/ "Geek for Geeks PCA"), [[2]])

Generally speaking, there are three key steps to PCA:  
1. Standardize the testing data.
2. Generate a covariance matrix.
3. Compute eigenvalues/eigenvectors of covariance matrix to identify the principle components.  

We'll skip to calculating the covariance matrix.

##### 2.2.1. Covariance Matrix
To generate the covariance matrix, we perform matrix multiplication on itself transposed.
i.e., $C = A A^T$.  
What you'll notice, however, is that we are performing the mutliplication for dimensionalities $N^2 \times M$ and $M \times N^2$\
which results in a dimensionality of $N^2 \times N^2$, which if we want to perform any more matrix manipulation will be too computationally large! 

Thankfully, we can also perform matrix manipulations to achieve desired results with a 
lower dimensionality matrix defined as: $L = A^T A$, which is a matrix of $M \times M$ dimensionality. 

In [85]:
L = np.dot(A.T, A)

**Optional:** You can test for your self to compare the computational resource\
required to calculate covariance-matrix $C$. Run the following code and see the\
time it takes for the notebook.

Since we are performing a calculation that results in a $N^2 \times N^2$ dimensionality  matrix,  
we are performing calculations for $N^4$ elements ($N^4 = 128^4 = 16,384$)!

In [86]:
# Optional
C = np.dot(A, A.T)

#### 2.2.2. Computing Eigenvalues/Eigenvectors
text about deriving eigenvalues and eigenvectors from covariance matrix,
and pulling first k eigenvectors from the eigenvalue pair.

In [176]:
# Top eigenfaces used
M_prime = 50
lambda_, u = np.linalg.eig(L)

# Sort eigenvectors by eigenvalues largest to smallest, and select first M'.
eigenpair = sorted([(lambda_[i], u[i]) for i in range(M)])[:M_prime:-1]
u = np.array([eigenpair[i][1] for i in range(M_prime)])

In [ ]:
# Optional
u_proj = np.dot(u, A.T)
display_eigenfaces = plt.figure(figsize=(5, 15))
for i in range(M_prime):
    display_eigenfaces.add_subplot(10, 5, i + 1),plt.title(i),plt.imshow(u_proj[i].reshape(N, N))

### 3.1. Classification
Required to project new image onto face-space, one eigenface at a time.  
Per eigenface, formula defined as: $w_k = u_k^T (\Gamma - \psi)$.  
For completion in one operation: $W = u^T (\Gamma - \psi)$ 

In [183]:
def get_weights(image):
    global M_prime, u, psi

    W = []
    for k in range(M_prime):
        normalized_image = image - psi 
        W.append(np.dot(u[k].reshape(M, 1).T, normalized_image.T).T)
    return np.array(W)

def regenerate_image(image):
    global psi
    W = get_weights(image)
    print(W.shape)

print(get_weights(A_0[3])[0].shape)

(16384, 1)
